## Valuation Model 

# Modelo Interativo de Valuation com Monte Carlo

Bem-vindo a este notebook interativo para avaliação de empresas, desenvolvido em Python. Este modelo utiliza duas abordagens principais para a análise financeira:

1. **Projeção Determinística:**  
   Gera os demonstrativos financeiros – como o Demonstrativo de Resultados (DRE), o Fluxo de Caixa e o Balanço Patrimonial – com base em premissas fixas. A partir desses dados, calcula-se o Valor da Empresa (EV) e o Preço da Ação. A projeção incorpora premissas como o crescimento dos clientes, o churn, os custos, as despesas, os salários e outros fatores importantes para a operação de uma empresa de pequeno porte.

2. **Simulação de Monte Carlo:**  
   Considerando a incerteza dos parâmetros críticos (como WACC, taxa de crescimento, CAPEX, churn e margem), o modelo realiza múltiplas simulações para gerar distribuições dos resultados. Dessa forma, é possível analisar a variabilidade do EV e do aporte (necessidade de investimento) e realizar uma análise de sensibilidade, comparando cenários pessimista, base e otimista.

## Recursos do Notebook

- **Interface Interativa:**  
  Utiliza o ipywidgets para permitir que o usuário insira, ajuste e combine premissas e parâmetros financeiros de forma intuitiva. As entradas estão organizadas em abas (Empresa, Valuation e Outros) para facilitar a configuração dos dados.

- **Exportação dos Resultados:**  
  O modelo possibilita a exportação dos demonstrativos financeiros gerados para arquivos CSV, permitindo análises adicionais e integração com outras ferramentas.

- **Análise de Sensibilidade e Cenários:**  
  O notebook gera gráficos que mostram a distribuição do EV, a distribuição do aporte, um gráfico de sensibilidade (tornado) que destaca os parâmetros que mais influenciam o resultado, e gráficos comparativos dos diferentes cenários (Pessimista, Base e Otimista).

## Como Utilizar

1. Ajuste os parâmetros da empresa (como ano base, clientes iniciais, capital, ativos, margem, ticket, etc.) na aba "Empresa".
2. Configure os parâmetros financeiros do valuation (WACC, crescimento, CAPEX, churn, crescimento perpétuo, depreciação, imposto, capital de giro) na aba "Valuation".
3. Defina as configurações para a simulação Monte Carlo (frequência de compra, número de simulações e período de análise) na aba "Outros".
4. Utilize os botões:
   - **Projeções:** para gerar e visualizar os demonstrativos financeiros e os resultados da projeção determinística.
   - **Monte Carlo:** para executar a simulação e visualizar os gráficos de distribuição e análise de sensibilidade.
   - **Exportar Resultados:** para salvar os demonstrativos financeiros gerados em arquivos CSV.

Este modelo foi projetado para ajudar na tomada de decisão ao avaliar o potencial e os riscos de investimentos em empresas de pequeno porte, oferecendo uma visão abrangente dos resultados financeiros sob diferentes cenários.

---

#### Instalação das bibliotecas necessárias para o código

In [1]:
# Instalação das bibliotecas necessárias
!pip install numpy pandas matplotlib ipywidgets

#### Utilize o botão abaixo para verificar se ipywidgets foi instalado corretamente

In [2]:
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline

##############################################################################
# TESTE MÍNIMO DO ipywidgets
##############################################################################
botao_teste = widgets.Button(description="Botão Teste")

def ao_clicar_teste(_):
    print("Funcionando! ipywidgets OK no seu ambiente.")

botao_teste.on_click(ao_clicar_teste)
display(botao_teste)

Button(description='Botão Teste', style=ButtonStyle())

In [4]:
import os
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline

# Crescimento de salários executivos fixo em 3% a.a.
CRESC_SALARIOS = 0.03

##############################################################################
# Função de amostragem triangular robusta (reordena low/high se necessário)
##############################################################################
def tri_amostra(base, lf=0.8, hf=1.2, minv=None, maxv=None):
    naive_low  = base * lf
    naive_high = base * hf
    low  = min(naive_low, naive_high)
    high = max(naive_low, naive_high)
    if minv is not None:
        low  = max(low,  minv)
        high = max(high, minv)
    if maxv is not None:
        low  = min(low,  maxv)
        high = min(high, maxv)
    if base < low:
        base = low
    if base > high:
        base = high
    if low >= high:
        high = low + 1e-9
    return np.random.triangular(low, base, high)

##############################################################################
# Projeção Determinística
##############################################################################
def projecao_deterministica(dados, T):
    base_year    = dados['base_year']
    clientes     = dados['initial_clients']
    acoes        = dados['shares']
    capital_inic = dados['initial_capital']
    ativos_nc    = dados['non_current_assets']
    margem       = dados['margin']       # ex.: 0.80
    ticket       = dados['ticket']
    emp_fixo     = dados['fixed_emp']
    emp_por_cli  = dados['emp_per_client']
    desp_fixas0  = dados['fixed_exp']
    desp_var     = dados['var_exp']
    sal_exec0    = dados['exec_salaries']
    
    wacc         = dados['wacc']
    growth_rate  = dados['growth_rate']
    capex_cli    = dados['capex']
    churn        = dados['churn']
    g_perp       = dados['g_perp']
    
    freq         = dados['freq']
    tax_rate     = dados['tax']
    wc_frac      = dados['wc']
    depr_rate    = dados['depr']
    
    caixa        = capital_inic
    min_caixa    = caixa
    lucro_acum   = 0.0
    ativos_atual = ativos_nc
    wc_ant       = 150000.0
    
    DRE = {"Clientes": [], "Funcionários": [], "Receita": [], "Lucro Bruto": [],
           "Salários Exec": [], "Despesas Fixas": [], "Custo Variável": [],
           "EBITDA": [], "Depreciação": [], "EBIT": [], "Impostos": [], "Lucro Líquido": []}
    FC = {"Lucro Líquido": [], "Depreciação": [], "CAPEX": [],
          "Δ Capital Giro": [], "Fluxo de Caixa Livre": []}
    BAL = {"Clientes": [], "Funcionários": [], "Ativos Não Circ.": [],
           "Capital de Giro": [], "Caixa": [], "Patrimônio Líquido": []}
    
    FCFF_list = []
    
    for ano in range(1, T+1):
        sal_exec_ano = sal_exec0 * (1 + CRESC_SALARIOS)**(ano-1)
        desp_fixa_ano = desp_fixas0 * (1 + growth_rate/2)**(ano-1)
    
        novos_clientes = clientes * growth_rate
        perdidos = clientes * churn
    
        funcs = emp_fixo + emp_por_cli * clientes
        receita = clientes * freq * 12 * ticket
        luc_bruto = receita * margem
        custo_var = funcs * (desp_var * (1 + growth_rate/2)**(ano-1))
    
        ebitda = luc_bruto - custo_var - sal_exec_ano - desp_fixa_ano
        dep = depr_rate * ativos_atual
        ebit = ebitda - dep
        impostos = ebit * tax_rate if ebit > 0 else 0
        ll = ebit - impostos
    
        cap_giro = receita * wc_frac
        delta_cg = cap_giro - wc_ant
        wc_ant = cap_giro
    
        capex_ano = novos_clientes * capex_cli
        fcf = ll + dep - capex_ano - delta_cg
        FCFF_list.append(fcf)
    
        ativos_atual += capex_ano - dep
        caixa += fcf
        if caixa < min_caixa:
            min_caixa = caixa
        lucro_acum += ll
    
        DRE["Clientes"].append(int(round(clientes)))
        DRE["Funcionários"].append(int(round(funcs)))
        DRE["Receita"].append(int(round(receita)))
        DRE["Lucro Bruto"].append(int(round(luc_bruto)))
        DRE["Salários Exec"].append(int(round(sal_exec_ano)))
        DRE["Despesas Fixas"].append(int(round(desp_fixa_ano)))
        DRE["Custo Variável"].append(int(round(custo_var)))
        DRE["EBITDA"].append(int(round(ebitda)))
        DRE["Depreciação"].append(int(round(dep)))
        DRE["EBIT"].append(int(round(ebit)))
        DRE["Impostos"].append(int(round(impostos)))
        DRE["Lucro Líquido"].append(int(round(ll)))
    
        FC["Lucro Líquido"].append(int(round(ll)))
        FC["Depreciação"].append(int(round(dep)))
        FC["CAPEX"].append(int(round(capex_ano)))
        FC["Δ Capital Giro"].append(int(round(delta_cg)))
        FC["Fluxo de Caixa Livre"].append(int(round(fcf)))
    
        BAL["Clientes"].append(int(round(clientes)))
        BAL["Funcionários"].append(int(round(funcs)))
        BAL["Ativos Não Circ."].append(int(round(ativos_atual)))
        BAL["Capital de Giro"].append(int(round(cap_giro)))
        BAL["Caixa"].append(int(round(caixa)))
        BAL["Patrimônio Líquido"].append(int(round(capital_inic + lucro_acum)))
    
        clientes = clientes + novos_clientes - perdidos
    
    TV = 0
    if wacc > g_perp and wacc > 0:
        TV = FCFF_list[-1] * (1 + g_perp) / (wacc - g_perp)
    EV = 0
    for i, fcf in enumerate(FCFF_list):
        EV += fcf / ((1 + wacc)**(i+1))
    EV += TV / ((1 + wacc)**len(FCFF_list))
    preco_acao = EV / acoes
    
    idx = [str(dados['base_year'] + i) for i in range(0, T)]
    df_DRE = pd.DataFrame(DRE, index=idx)
    df_FC  = pd.DataFrame(FC, index=idx)
    df_BAL = pd.DataFrame(BAL, index=idx)
    
    prob_fechar_det = 1.0 if min_caixa < 0 else 0.0
    
    return {
        "DRE": df_DRE,
        "Fluxo de Caixa": df_FC,
        "Balanco": df_BAL,
        "EV": int(round(EV, 0)),
        "PrecoAcao": int(round(preco_acao, 0)),
        "prob_fechar_det": prob_fechar_det
    }

##############################################################################
# Simulação de Monte Carlo
##############################################################################
def simulacao_monte_carlo(dados, iteracoes, anos):
    init_clients = dados['initial_clients']
    wacc_base = dados['wacc']
    grow_base = dados['growth_rate']
    churn_base = dados['churn']
    capex_base = dados['capex']
    margin_base = dados['margin']
    freq = dados['freq']
    tax_rate = dados['tax']
    wc_perc = dados['wc']
    dep_rate = dados['depr']
    fix_emp = dados['fixed_emp']
    emp_cli = dados['emp_per_client']
    var_exp = dados['var_exp']
    sal_exec0 = dados['exec_salaries']
    fix_exp0 = dados['fixed_exp']
    capital_inic = dados['initial_capital']
    ativos_nc = dados['non_current_assets']
    perp_base = dados['g_perp']
    ticket_base = dados['ticket']
    
    ev_array = np.zeros(iteracoes)
    aporte_array = np.zeros(iteracoes)
    encerrados = np.zeros(iteracoes, dtype=bool)
    
    for i in range(iteracoes):
        wacc_sim = tri_amostra(wacc_base, lf=0.8, hf=1.2, minv=0.0001)
        grow_sim = tri_amostra(grow_base, lf=0.8, hf=1.2)  
        churn_sim = tri_amostra(churn_base, lf=0.8, hf=1.2, minv=0, maxv=1)
        capex_sim = tri_amostra(capex_base, lf=0.8, hf=1.2, minv=0)
        marg_sim = tri_amostra(margin_base, lf=0.8, hf=1.2, minv=0, maxv=1)
        if wacc_sim <= perp_base:
            wacc_sim = perp_base + 0.001
        
        c = init_clients
        caixa = capital_inic
        min_cash = caixa
        ativos = ativos_nc
        wc_ant = 150000
        fcfs = []
        
        for ano in range(1, anos+1):
            sal_ex_ano = sal_exec0 * (1 + CRESC_SALARIOS)**(ano-1)
            fix_exp_ano = fix_exp0 * (1 + grow_sim/2)**(ano-1)
    
            novos = c * grow_sim
            perd = c * churn_sim
            receita = c * freq * 12 * ticket_base
            luc_bruto = receita * marg_sim
            total_emp = fix_emp + emp_cli * c
            custo_var = total_emp * (var_exp * (1 + grow_sim/2)**(ano-1))
            ebitda = luc_bruto - custo_var - sal_ex_ano - fix_exp_ano
            dep = dep_rate * ativos
            ebt = ebitda - dep
            imp = ebt * tax_rate if ebt > 0 else 0
            ll = ebt - imp
            wc_req = receita * wc_perc
            dwc = wc_req - wc_ant
            wc_ant = wc_req
            capex_ano = novos * capex_sim
            fcf = ll + dep - capex_ano - dwc
            fcfs.append(fcf)
    
            caixa += fcf
            if caixa < min_cash:
                min_cash = caixa
            c = c + novos - perd
            ativos += capex_ano - dep
        
        TV = 0
        if wacc_sim > perp_base:
            TV = fcfs[-1] * (1 + perp_base) / (wacc_sim - perp_base)
        ev = 0
        for idx, fcf in enumerate(fcfs):
            ev += fcf / ((1 + wacc_sim)**(idx+1))
        ev += TV / ((1 + wacc_sim)**anos)
    
        aporte = 0
        if min_cash < 0:
            aporte = -min_cash
            encerrados[i] = True
        
        ev_array[i] = ev
        aporte_array[i] = aporte
    
    prob_enc = encerrados.mean()
    return ev_array, aporte_array, prob_enc

##############################################################################
# Cenários
##############################################################################
cenarios = {
    "Pessimista": {"wacc": +0.05, "growth_rate": -0.10, "churn": +0.05, "margin": -0.05},
    "Base": {},
    "Otimista": {"wacc": -0.03, "growth_rate": +0.10, "churn": -0.02, "margin": +0.05}
}

##############################################################################
# Interface (todos os inputs em abas separadas)
##############################################################################
def interface_valuation():
    out = widgets.Output()
    
    # ABA Empresa
    scenario_dropdown = widgets.Dropdown(options=["Pessimista", "Base", "Otimista"],
                                         value="Base", description="Cenário:")
    base_year_in = widgets.IntText(value=2025, description="Ano Base")
    init_clients_in = widgets.IntText(value=1000, description="Clientes Inic.")
    shares_in = widgets.IntText(value=1000, description="Qtd Ações")
    init_cap_in = widgets.IntText(value=20000, description="Capital Inicial")
    nc_assets_in = widgets.IntText(value=12000, description="Ativos N/Circ")
    margin_in = widgets.FloatText(value=80, description="Margem(%)")
    ticket_in = widgets.FloatText(value=100, description="Ticket")
    fix_emp_in = widgets.FloatText(value=1, description="Func Fixos")
    emp_cli_in = widgets.FloatText(value=0.001, description="Func/Cliente")
    fix_exp_in = widgets.IntText(value=2000, description="Desp Fixas")
    var_exp_in = widgets.IntText(value=1200, description="Desp Var/Func")
    exec_sal_in = widgets.IntText(value=48000, description="Sal Exec")
    box_empresa = widgets.VBox([scenario_dropdown, base_year_in, init_clients_in, shares_in,
                                init_cap_in, nc_assets_in, margin_in, ticket_in,
                                fix_emp_in, emp_cli_in, fix_exp_in, var_exp_in, exec_sal_in])
    
    # ABA Valuation
    wacc_in = widgets.FloatText(value=13.5, description="WACC(%)")
    growth_in = widgets.FloatText(value=13, description="Cresc(%)")
    capex_in = widgets.FloatText(value=1, description="CAPEX")
    churn_in = widgets.FloatText(value=3, description="Churn(%)")
    perp_in = widgets.FloatText(value=2.5, description="CrescPerp(%)")
    depr_in = widgets.FloatText(value=10, description="Depr(%)")
    imp_in = widgets.FloatText(value=12.5, description="Imposto(%)")
    wc_in = widgets.FloatText(value=10, description="WC(%)")
    box_val = widgets.VBox([wacc_in, growth_in, capex_in, churn_in,
                            perp_in, depr_in, imp_in, wc_in])
    
    # ABA Outros
    freq_in = widgets.FloatText(value=0.08333, description="Freq(Mês)")
    sims_in = widgets.IntText(value=10000, description="Simulações")
    years_in = widgets.IntText(value=5, description="Anos Análise")
    box_outros = widgets.VBox([freq_in, sims_in, years_in])
    
    tabs = widgets.Tab(children=[box_empresa, box_val, box_outros])
    tabs.set_title(0, "Empresa")
    tabs.set_title(1, "Valuation")
    tabs.set_title(2, "Outros")
    
    # Botões: Projeções, Monte Carlo e Exportar Resultados
    bot_det = widgets.Button(description="Projeções", button_style='info')
    bot_mc = widgets.Button(description="Monte Carlo", button_style='success')
    bot_exp = widgets.Button(description="Exportar Resultados", button_style='warning')
    
    def rodar_deterministico(_):
        out.clear_output()
        with out:
            T = years_in.value
            dados = {
                'base_year': base_year_in.value,
                'initial_clients': float(init_clients_in.value),
                'shares': float(shares_in.value),
                'initial_capital': float(init_cap_in.value),
                'non_current_assets': float(nc_assets_in.value),
                'margin': float(margin_in.value)/100.0,
                'ticket': float(ticket_in.value),
                'fixed_emp': float(fix_emp_in.value),
                'emp_per_client': float(emp_cli_in.value),
                'fixed_exp': float(fix_exp_in.value),
                'var_exp': float(var_exp_in.value),
                'exec_salaries': float(exec_sal_in.value),
                'wacc': float(wacc_in.value)/100.0,
                'growth_rate': float(growth_in.value)/100.0,
                'capex': float(capex_in.value),
                'churn': float(churn_in.value)/100.0,
                'g_perp': float(perp_in.value)/100.0,
                'depr': float(depr_in.value)/100.0,
                'tax': float(imp_in.value)/100.0,
                'wc': float(wc_in.value)/100.0,
                'freq': float(freq_in.value)
            }
            cen = scenario_dropdown.value
            if cen != "Base":
                aj = cenarios[cen]
                for k, v in aj.items():
                    dados[k] += v
            dados['margin'] = max(0, min(dados['margin'], 1))
            dados['churn'] = max(0, min(dados['churn'], 1))
            dados['wacc'] = max(0.0001, dados['wacc'])
            if dados['wacc'] <= dados['g_perp']:
                dados['wacc'] = dados['g_perp'] + 0.001
            
            proj = projecao_deterministica(dados, T)
            idx = [str(dados['base_year'] + i) for i in range(0, T)]
            proj["DRE"].index = idx
            proj["Fluxo de Caixa"].index = idx
            proj["Balanco"].index = idx
            
            print("=== Projeções (Determinística) ===")
            print("\nDRE:")
            display(proj["DRE"])
            print("\nFluxo de Caixa:")
            display(proj["Fluxo de Caixa"])
            print("\nBalanço Patrimonial:")
            display(proj["Balanco"])
            print(f"\nValor da Empresa (EV): {proj['EV']} (R$)")
            print(f"Preço da Ação: {proj['PrecoAcao']} (R$)")
            print(f"Probabilidade de Encerramento (Determinística): {proj['prob_fechar_det']*100:.1f}%")
            
            # Armazena os resultados globais para exportação
            global resultados_export
            resultados_export = proj

    def rodar_mc(_):
        out.clear_output()
        with out:
            T = years_in.value
            sims = sims_in.value
            dados = {
                'base_year': base_year_in.value,
                'initial_clients': float(init_clients_in.value),
                'shares': float(shares_in.value),
                'initial_capital': float(init_cap_in.value),
                'non_current_assets': float(nc_assets_in.value),
                'margin': float(margin_in.value)/100.0,
                'ticket': float(ticket_in.value),
                'fixed_emp': float(fix_emp_in.value),
                'emp_per_client': float(emp_cli_in.value),
                'fixed_exp': float(fix_exp_in.value),
                'var_exp': float(var_exp_in.value),
                'exec_salaries': float(exec_sal_in.value),
                'wacc': float(wacc_in.value)/100.0,
                'growth_rate': float(growth_in.value)/100.0,
                'capex': float(capex_in.value),
                'churn': float(churn_in.value)/100.0,
                'g_perp': float(perp_in.value)/100.0,
                'depr': float(depr_in.value)/100.0,
                'tax': float(imp_in.value)/100.0,
                'wc': float(wc_in.value)/100.0,
                'freq': float(freq_in.value)
            }
            cen = scenario_dropdown.value
            if cen != "Base":
                aj = cenarios[cen]
                for k, v in aj.items():
                    dados[k] += v
            dados['margin'] = max(0, min(dados['margin'], 1))
            dados['churn'] = max(0, min(dados['churn'], 1))
            dados['wacc'] = max(0.0001, dados['wacc'])
            if dados['wacc'] <= dados['g_perp']:
                dados['wacc'] = dados['g_perp'] + 0.001

            ev_array, aporte_array, prob_enc = simulacao_monte_carlo(dados, sims, T)
    
            p5_ev  = np.percentile(ev_array, 5)
            p50_ev = np.percentile(ev_array, 50)
            p95_ev = np.percentile(ev_array, 95)
    
            plt.figure(figsize=(6,4))
            ev_k = ev_array/1e3
            plt.hist(ev_k, bins=30, color='skyblue', edgecolor='black')
            plt.axvline(p5_ev/1e3, color='green', linestyle='--', label=f"5º Percentil = {p5_ev/1e3:,.0f}k")
            plt.axvline(p50_ev/1e3, color='blue', linestyle='--', label=f"Mediana = {p50_ev/1e3:,.0f}k")
            plt.axvline(p95_ev/1e3, color='green', linestyle='--', label=f"95º Percentil = {p95_ev/1e3:,.0f}k")
            plt.title("Distribuição do Valor da Empresa (EV) [R$ mil]")
            plt.xlabel("EV (R$ mil)")
            plt.ylabel("Frequência")
            plt.legend()
            plt.show()
    
            p5_ap  = np.percentile(aporte_array, 5)
            p50_ap = np.percentile(aporte_array, 50)
            p95_ap = np.percentile(aporte_array, 95)
            plt.figure(figsize=(6,4))
            ap_k = aporte_array/1e3
            plt.hist(ap_k, bins=30, color='darkgreen', edgecolor='black')
            plt.axvline(p5_ap/1e3, color='green', linestyle='--', label=f"5º Percentil = {p5_ap/1e3:,.0f}k")
            plt.axvline(p50_ap/1e3, color='blue', linestyle='--', label=f"Mediana = {p50_ap/1e3:,.0f}k")
            plt.axvline(p95_ap/1e3, color='green', linestyle='--', label=f"95º Percentil = {p95_ap/1e3:,.0f}k")
            plt.title("Distribuição da Necessidade de Investimento [R$ mil]")
            plt.xlabel("Aporte (R$ mil)")
            plt.ylabel("Frequência")
            plt.legend()
            plt.show()
    
            from copy import deepcopy
            def single_run_ev(dd, T_):
                return projecao_deterministica(dd, T_)["EV"]
            EV_base = single_run_ev(dados, T)
            def vary_param(nome, base_val):
                lf = 0.8
                hf = 1.2
                lv = base_val * lf
                hv = base_val * hf
                if nome == "wacc" and lv <= dados['g_perp']:
                    lv = dados['g_perp'] + 0.001
                if nome in ["Margem", "Churn"]:
                    lv = max(0, min(lv, 1))
                    hv = max(0, min(hv, 1))
                dd_low = deepcopy(dados)
                dd_high = deepcopy(dados)
                if nome == "wacc":
                    dd_low["wacc"] = lv
                    dd_high["wacc"] = hv
                elif nome == "Crescimento":
                    dd_low["growth_rate"] = lv
                    dd_high["growth_rate"] = hv
                elif nome == "Churn":
                    dd_low["churn"] = lv
                    dd_high["churn"] = hv
                elif nome == "Margem":
                    dd_low["margin"] = lv
                    dd_high["margin"] = hv
                elif nome == "CAPEX":
                    dd_low["capex"] = lv
                    dd_high["capex"] = hv
                ev_low = single_run_ev(dd_low, T)
                ev_high = single_run_ev(dd_high, T)
                delta_low = max((ev_low - EV_base) / 1e3, 0)
                delta_high = max((ev_high - EV_base) / 1e3, 0)
                return (nome, delta_low, delta_high)
    
            param_list = [
                ("wacc", dados["wacc"]),
                ("Crescimento", dados["growth_rate"]),
                ("Churn", dados["churn"]),
                ("Margem", dados["margin"]),
                ("CAPEX", dados["capex"])
            ]
            arr_sens = [vary_param(nm, val) for nm, val in param_list]
            arr_sens.sort(key=lambda x: x[2] - x[1], reverse=True)
    
            plt.figure(figsize=(6,4))
            nomes = [x[0] for x in arr_sens]
            ev_low_vals = [x[1] for x in arr_sens]
            ev_high_vals = [x[2] for x in arr_sens]
            y_pos = np.arange(len(nomes))
            for i, nm in enumerate(nomes):
                plt.hlines(y=i, xmin=ev_low_vals[i], xmax=ev_high_vals[i], color='darkblue', linewidth=10)
            plt.axvline(0, color='black', linestyle='dotted', label="EV Base (Deslocado para 0)")
            plt.yticks(y_pos, nomes)
            plt.xlabel("Impacto no Valor da Empresa (R$ mil) [mín=0]")
            plt.title("Análise de Sensibilidade (Tornado)")
            plt.legend(loc='upper right')
            plt.xlim(left=0)
            plt.gca().invert_yaxis()
            plt.tight_layout()
            plt.show()
    
            from copy import deepcopy
            def scenario_mc_ev(base_info, overrides, T_):
                dd = deepcopy(base_info)
                for k, v in overrides.items():
                    dd[k] += v
                ev_arr2, _, _ = simulacao_monte_carlo(dd, sims, T_)
                return np.mean(ev_arr2)
            scen_list = ["Pessimista", "Base", "Otimista"]
            scen_vals = []
            for sc in scen_list:
                sc_aj = cenarios[sc]
                ev_medio = scenario_mc_ev(dados, sc_aj, T)
                scen_vals.append(ev_medio / 1e3)
            plt.figure(figsize=(6,4))
            plt.bar(scen_list, scen_vals, color=["red", "gray", "green"])
            plt.title("Valor da Empresa (EV) nos Cenários (Monte Carlo) [R$ mil]")
            plt.xlabel("Cenário")
            plt.ylabel("EV Médio (R$ mil)")
            plt.show()
    
    # Função para exportar os resultados da projeção (armazenados globalmente em 'resultados_export')
    def rodar_exportar(_):
        out.clear_output()
        with out:
            if 'resultados_export' not in globals():
                print("Nenhum resultado foi gerado. Execute 'Projeções' primeiro.")
                return
            diretorio = "/Users/visol/Documents/PythonDocs/"
            if not os.path.exists(diretorio):
                os.makedirs(diretorio)
                print(f"Diretório '{diretorio}' criado.")
            try:
                path_dre = os.path.join(diretorio, "DRE_projecao.csv")
                path_fc  = os.path.join(diretorio, "Fluxo_de_Caixa.csv")
                path_bal = os.path.join(diretorio, "Balanco_projecao.csv")
                resultados_export["DRE"].to_csv(path_dre, index=True)
                resultados_export["Fluxo de Caixa"].to_csv(path_fc, index=True)
                resultados_export["Balanco"].to_csv(path_bal, index=True)
                print("Arquivos exportados com sucesso!")
                print(f"DRE exportado para: {path_dre}")
                print(f"Fluxo de Caixa exportado para: {path_fc}")
                print(f"Balanco exportado para: {path_bal}")
            except Exception as e:
                print("Ocorreu um erro ao exportar os arquivos:", e)
    
    bot_det.on_click(rodar_deterministico)
    bot_mc.on_click(rodar_mc)
    bot_exp.on_click(rodar_exportar)
    
    display(widgets.VBox([tabs, widgets.HBox([bot_det, bot_mc, bot_exp]), out]))

interface_valuation()

#### **Disclaimer de Propriedade Intelectual**

Todo o conteúdo deste notebook, incluindo (mas não se limitando a) código-fonte, textos, comentários, funções, classes e quaisquer outras criações intelectuais aqui presentes, está protegido pelas leis de propriedade intelectual. A reprodução, distribuição, exibição ou execução pública, bem como a adaptação, tradução, arranjo, compilação, modificação ou qualquer forma de utilização deste conteúdo, total ou parcial, sem a devida autorização, constitui violação dos direitos do titular.

Qualquer uso não autorizado deste material pode resultar em sanções civis, administrativas e/ou penais, conforme a legislação vigente. Se desejar obter permissão para utilização, reprodução ou distribuição do conteúdo aqui apresentado, entre em contato com o detentor dos direitos autorais.

---

**José Paulo Vicente De Sousa Lima**

#### Contato
josepvslima@outlook.com